# Testing GPT2 on spongebob scripts 

Credit to the work done by Mikhail Gaerlan and Max Woolf

# Installation and Asserting Version Requirements

In [1]:

from platform import python_version
print(python_version())

3.7.10


In [1]:

%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from glob import glob



TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [3]:
!nvidia-smi

Tue Apr  6 21:19:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading GPT2


* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

In [4]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 690Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.57Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 707Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:35, 5.23Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 573Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.40Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.47Mit/s]


# Uploading text file for training

In [ ]:
# import os
# os.chdir("drive/MyDrive/w266 Final Project Resources")
# os.listdir()



In [9]:

file_name = "spongebob_updated (2).txt"



In [10]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
sess = gpt2.start_tf_sess()


In [ ]:
#gpt2.load_gpt2(sess)

In [11]:

gpt2.finetune(sess,
              dataset=file_name,
              restore_from ='fresh',
              model_name='124M',
              steps=500,
              print_every=50,
              sample_every=500,
              save_every=100,
             
              batch_size=2

             
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:10<00:00, 10.49s/it]


dataset has 2084091 tokens
Training...
[50 | 124.32] loss=2.57 avg=2.57
[100 | 243.02] loss=2.55 avg=2.56
Saving checkpoint/run1/model-100
[150 | 363.92] loss=2.59 avg=2.57
[200 | 482.59] loss=2.41 avg=2.53
Saving checkpoint/run1/model-200
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[250 | 602.94] loss=2.28 avg=2.48
[300 | 721.64] loss=2.14 avg=2.42
Saving checkpoint/run1/model-300
[350 | 842.08] loss=2.26 avg=2.40
[400 | 960.81] loss=2.03 avg=2.35
Saving checkpoint/run1/model-400
[450 | 1081.32] loss=1.90 avg=2.30
[500 | 1200.02] loss=2.03 avg=2.27
Saving checkpoint/run1/model-500


In [13]:
gpt2.copy_checkpoint_to_gdrive(run_name='run2')

FileNotFoundError: ignored

In [8]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


In [9]:
for i in range(406,500):
  gpt2.generate(sess,
                model_name='124M',
                batch_size=1,
                prefix = "<Start Ep>",
      
                temperature=.7,
                truncate = "<End Ep>",
                include_prefix = True,
                destination_path=f'/content/drive/MyDrive/GPT2SIMPLE/class_false/spongebob_{i}.txt')


In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')
        

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below, comented out for safety:

In [ ]:
#!kill -9 -1